In [96]:
import rasa_core
from rasa_core.agent import Agent
from bokeh.io import output_notebook
agent = Agent.load("models/dialogue_embed")
# agent = Agent.load("models/dialogue_embed_before_norm_loss")
output_notebook()

INFO:tensorflow:Restoring parameters from models/dialogue_embed/policy_0_EmbeddingPolicy/tensorflow_embedding.ckpt


Loading BokehJS ...

In [97]:
rasa_core.__version__

'0.13.0a4'

In [98]:
policy = agent.policy_ensemble.policies[0]
interpreter = agent.interpreter
print(policy.sparse_attention)

False


In [99]:
domain = agent.domain


In [100]:
from rasa_core.training.structures import StoryGraph
from rasa_core.training.dsl import StoryFileReader
from rasa_core.trackers import DialogueStateTracker
from rasa_core.channels import UserMessage
import numpy as np
np.set_printoptions(precision=3, suppress=True)

def embedding_from_tracker(story, bot=False, predict_only=False):
    reader = StoryFileReader(domain, interpreter)
    story = story.split('\n')
    story_steps = reader.process_lines(story)
#     story_graph = StoryGraph(story_steps)
    tracker = DialogueStateTracker(
            1,
            domain.slots
    )
    tracker._reset()
    for step in story_steps:
        events = step.explicit_events(domain)
        for event in events[:-1]:
            tracker.update(event)

    if predict_only:
        return policy.predict_action_probabilities(tracker, domain)

    data_X = policy.featurizer.create_X([tracker], domain)
    session_data = policy._create_tf_session_data(domain, data_X)
    all_Y_d_x = np.stack([session_data.all_Y_d for _ in range(session_data.X.shape[0])])
    feed_dict={policy.a_in: session_data.X,
               policy.b_in: all_Y_d_x,
               policy.c_in: session_data.slots,
               policy.b_prev_in: session_data.previous_actions,
               policy._dialogue_len: session_data.X.shape[1],
               policy._x_for_no_intent_in: session_data.x_for_no_intent,
               policy._y_for_no_action_in: session_data.y_for_no_action,
               policy._y_for_action_listen_in: session_data.y_for_action_listen}

    dial_vec = policy.session.run(policy.dial_embed, feed_dict=feed_dict)
    bot_vec = policy.session.run(policy.bot_embed, feed_dict=feed_dict)

    no_skip_gate = policy.session.run(policy.copy_attn_debug, feed_dict=feed_dict)
    attn_embed = policy.session.run(policy.attn_embed, feed_dict=feed_dict)
    rnn_embed = policy.session.run(policy.rnn_embed, feed_dict=feed_dict)
    probs = policy.session.run(policy.alignment_history, feed_dict=feed_dict)
    time_mask = policy.session.run(policy.all_time_masks, feed_dict=feed_dict)
    topics = policy.session.run(policy.topics, feed_dict=feed_dict)
    cell_state = policy.session.run(policy.cell_state, feed_dict=feed_dict)
    if probs.shape[2] == 2 * probs.shape[1]:
        user_probs = probs[0, -1, :int(probs.shape[2]/2)]
        bot_probs = probs[0, -1, int(probs.shape[2]/2):]
    else:
        user_probs = probs[0, -1, :]
        bot_probs = []

    sims = policy.session.run(policy.sim_op, feed_dict=feed_dict)
    if bot is False:
        sims1 = policy.predict_action_probabilities(tracker, domain)

    del tracker
#     print(np.shape(dial_vec))

    if bot is False:
        return (dial_vec[0,-1,:],
                no_skip_gate[0,-1,:],
                attn_embed[0,-1,:],
                rnn_embed[0,-1,:],
                [user_probs, bot_probs, time_mask[0,-1,:]],
                sims[0,-1,:],
                sims1,
                topics[0],
                cell_state[0]
               )
    else:
        return (dial_vec[0,-1,:],
                sims[0,-1,:],
                bot_vec[0,-1,:,:])
    
def make_prediction(story):
    dial, sim0, bot = embedding_from_tracker(story, bot=True)
    abs_vals = np.array([np.absolute(vec) for vec in bot])
    norms = np.apply_along_axis(np.linalg.norm, 1, bot)
    sims = np.dot(bot, dial)/norms
#     name = domain.action_for_index(np.argmax(sims)).name()
#     print(name)
    name = domain.action_names[np.argmax(sim0)]
#     print(name)
#     print('---')
#     ids = sim0.argsort()[::-1]
#     print(ids)
#     print(sims[ids])
    return '- ' + name, np.max(sim0)
#     print(dial.shape, bot.shape)

In [101]:
# print(domain.intents)
story_block = """* request_hotel
    - utter_ask_details
* chitchat
    - utter_chitchat
    - utter_ask_details
* inform{"enddate": "May 26th"}
    - slot{"enddate": "May 26th"}
    - utter_ask_startdate
* inform{"startdate": "next week"}
    - slot{"startdate": "next week"}
    - utter_ask_location
* correct{"startdate": "next week"}
    - slot{"startdate": "next week"}
    - utter_correct_startdate_hotel
    - utter_ask_location
* explain
    - utter_explain_location_hotel
    - utter_ask_location
* inform{"location": "rome"}
    - slot{"location": "rome"}
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* chitchat
    - utter_chitchat
    - utter_ask_price
* explain
    - utter_explain_price_hotel
    - utter_ask_price
* inform{"price": "expensive"}
    - utter_ask_people
* inform{"people": "4"}
    - utter_filled_slots
    - action_search_hotel
    - slot{"hotel": "hotel"}
    - utter_suggest_hotel
* chitchat
    - utter_chitchat
    - utter_suggest_hotel
* affirm
    - utter_happy"""

In [102]:
# story_block = """* request_hotel
#     - utter_ask_details
# * inform{"people": "4"}
#     - slot{"people": "4"}
#     - utter_ask_location
# * correct{"people": "2"}
#     - slot{"people": "2"}
#     - utter_correct_people_hotel
#     - utter_ask_location
# * chitchat
#     - utter_chitchat
#     - utter_ask_location
# * inform{"location": "rome"}
#     - slot{"location": "rome"}
#     - utter_ask_price
# * inform{"price": "expensive"}
#     - slot{"price": "expensive"}
#     - utter_ask_startdate
# * inform{"startdate": "next week", "enddate": "tomorrow"}
#     - slot{"startdate": "next week"}
#     - slot{"enddate": "tomorrow"}
#     - utter_filled_slots
#     - utter_search_hotel
#     - slot{"hotel": "hotel"}
#     - utter_suggest_hotel
# * correct{"enddate": "May 26th"}
#     - slot{"enddate": "May 26th"}
#     - utter_correct_enddate_hotel
#     - utter_search_hotel
#     - slot{"hotel": "hotel"}
#     - utter_suggest_hotel
# * correct{"location": "rome"}
#     - slot{"location": "rome"}
#     - utter_correct_location_hotel
#     - utter_search_hotel
#     - slot{"hotel": "hotel"}
#     - utter_suggest_hotel
# * did_that_work
#     - utter_worked_hotel
#     - utter_suggest_hotel
# * affirm
#     - utter_happy
# * chitchat"""

In [103]:
# story_block = """* request_hotel
#     - utter_ask_details
# * inform{"people": "2"}
#     - slot{"people": "2"}
#     - utter_ask_location
# * correct{"people": "2"}
#     - slot{"people": "2"}
#     - utter_correct_people_hotel
#     - utter_ask_location
# * explain
#     - utter_explain_location_hotel
#     - utter_ask_location
# * inform{"location": "paris"}
#     - utter_ask_price
# * inform{"price": "expensive"}
#     - utter_ask_startdate
# * inform{"startdate": "10.03.2018"}
#     - utter_ask_enddate
# * explain
#     - utter_explain_enddate_hotel
#     - utter_ask_enddate
# * explain
#     - utter_explain_enddate_hotel
#     - utter_ask_enddate
# * explain
#     - utter_explain_enddate_hotel
#     - utter_ask_enddate
# * explain
#     - utter_explain_enddate_hotel
#     - utter_ask_enddate
# * did_that_work
#     - utter_more_info_hotel
#     - utter_ask_enddate
# * inform{"enddate": "10.03.2018"}
#     - utter_filled_slots
#     - utter_search_hotel
#     - slot{"hotel": "hotel"}
#     - utter_suggest_hotel
# * did_that_work
#     - utter_worked_hotel
#     - utter_suggest_hotel
# * affirm
#     - utter_happy
# """

In [104]:
print(len(story_block.split('\n')))
cool_list = []
lines = story_block.split('\n')
for idx, line in enumerate(lines):
    if idx == len(lines)-1:
        continue
    if '*' in lines[idx+1] or 'slot{' in lines[idx+1]:
#     if 'slot{' in lines[idx+1]:
        continue
    sub_story = '\n'.join(lines[:idx+1])
    (dial, not_skip, attn, rnn, probs, sims, sims1, topic, cell_state
    ) = embedding_from_tracker("#generate story\n{}".format(sub_story))
#     print(np.max(sims), np.max(sims1))

    pred, sim = make_prediction("#generate story\n{}".format(sub_story))
    cool_list.append((lines[idx+1].strip(), dial, (pred, sim, sims), not_skip, attn[-20:], rnn,
                      probs, topic, cell_state))
#     embedding_from_tracker("#generate story\n{}".format(sub_story), predict_only=True)

print(len(cool_list))
# print(cool_list[0][4].shape)
#     break
# dial = embedding_from_tracker("#generate story\n{}".format(story_block)
# )
# print(dial.shape)
# print(len(story_block.split('\n')))
# print(probs[0, 6, :])

48
27


In [105]:
import copy

# cool_list0 = copy.deepcopy(cool_list)
cool_list[7]

('- utter_explain_location_hotel',
 array([-0.013,  0.001,  0.165,  0.127, -0.018,  0.019, -0.141, -0.041,
         0.246,  0.007, -0.087,  0.094, -0.005, -0.033,  0.17 ,  0.064,
        -0.055,  0.017, -0.189,  0.054], dtype=float32),
 ('- utter_explain_location_hotel',
  0.8260947,
  array([-0.039,  0.064,  0.076, -0.012, -0.105,  0.036, -0.058, -0.095,
         -0.526,  0.407, -0.323, -0.217, -0.235,  0.027, -0.085, -0.138,
          0.112, -0.138,  0.418,  0.133,  0.446,  0.826,  0.341,  0.267,
          0.027,  0.425,  0.139,  0.454,  0.15 ,  0.057, -0.156,  0.059,
         -0.333,  0.124,  0.488,  0.138, -0.035,  0.094, -0.07 , -0.12 ,
         -0.2  ,  0.126,  0.112,  0.079, -0.148, -0.123,  0.138],
        dtype=float32)),
 array([1., 1., 0., 0., 1.], dtype=float32),
 array([-0.02 , -0.015,  0.033,  0.034, -0.01 ,  0.046,  0.018, -0.027,
         0.019,  0.001,  0.011,  0.018,  0.002, -0.017,  0.014, -0.012,
        -0.005,  0.006,  0.006, -0.005], dtype=float32),
 array([ 0.00

In [106]:
# cool_list1= copy.deepcopy(cool_list)
cool_list[18]

('- utter_explain_price_hotel',
 array([ 0.062,  0.073,  0.097,  0.069,  0.042, -0.029, -0.157, -0.117,
         0.161,  0.013, -0.202,  0.105, -0.035,  0.046,  0.101,  0.05 ,
        -0.006,  0.03 , -0.126,  0.113], dtype=float32),
 ('- utter_explain_location_hotel',
  0.759085,
  array([-0.108,  0.091,  0.172,  0.037, -0.07 ,  0.067, -0.048, -0.115,
         -0.554,  0.108,  0.123, -0.324, -0.204,  0.007, -0.015, -0.121,
          0.203, -0.217,  0.504,  0.177,  0.559,  0.759,  0.294,  0.363,
          0.072,  0.651,  0.268,  0.525,  0.192,  0.128, -0.142,  0.166,
         -0.383,  0.165,  0.352,  0.052, -0.021,  0.281,  0.034, -0.078,
         -0.186,  0.148,  0.177,  0.103, -0.152, -0.199, -0.   ],
        dtype=float32)),
 array([1., 1., 0., 0., 1.], dtype=float32),
 array([ 0.015,  0.017, -0.003, -0.009,  0.02 ,  0.017,  0.029, -0.047,
        -0.014, -0.002, -0.02 ,  0.015, -0.009,  0.003, -0.013, -0.019,
         0.   ,  0.008,  0.019,  0.026], dtype=float32),
 array([ 0.047,  

In [58]:
# import matplotlib.pyplot as plt
# plt.plot(cool_list[15][-2], marker='x')

In [13]:
# plt.plot(cool_list[19][-2], marker='x')

In [33]:
from sklearn.decomposition import PCA

In [34]:
pca = PCA(n_components=2)
X = pca.fit_transform([ting[1] for ting in cool_list])

In [35]:
from bokeh.plotting import figure, show
import matplotlib.pyplot as plt

In [36]:
embed_dim = cool_list[0][1].shape[0]


In [37]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, Arrow, OpenHead

hover = HoverTool(tooltips=[("true action","@desc"), ("prediction", "@pred"),
                            ("not_skip_gate", "@not_skip")],names=['circle'])
p = figure( tools=[hover])

for i in range(1, len(X)):
    p.add_layout(Arrow(end=OpenHead(line_color="firebrick", size=10, line_width=2),
                       x_start=X[i-1,0], y_start=X[i-1,1],
                       x_end=X[i,0], y_end=X[i,1], line_color='gray'))

source = ColumnDataSource(data=dict(
    x=X[:,0],
    y=X[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=10, name='circle')
# p.line(X[:,0], X[:,1])

X1 = pca.transform([ting[4] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X1[:,0],
    y=X1[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='green', name='circle')

X2 = pca.transform([ting[5] for ting in cool_list])
source = ColumnDataSource(data=dict(
    x=X2[:,0],
    y=X2[:,1],
    desc=[ting[0] for ting in cool_list],
    pred = [ting[2][0] for ting in cool_list],
    not_skip = [ting[3][-1] for ting in cool_list]
))
p.circle('x','y', source=source, size=5, color='red', name='circle')

x = pca.transform(np.zeros((1,embed_dim)))
p.circle(x[0,0], x[0,1], size=5, color='magenta')
p.circle(X[0,0], X[0,1], size=10, color='green')
p.circle(X[-1,0], X[-1,1], size=10, color='red')


bokeh.models.renderers.GlyphRenderer(
    id='03e941ab-d215-4f7d-8f32-f819af542619',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='beef3413-9a47-45e0-acfe-01dc7eee71b0',
        callback=None,
        data={},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='4758660c-38fd-48b3-abce-2be50a0c6f3c',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='ea80e412-24a1-49f5-910e-7e419d7a3e8b',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.markers.Circle(
        id='d891f89a-ff4f-406b-83b7-0217e97184d3',
        angle={'value': 0.0, 'units': 'rad'},
        fill_alpha={'value': 1.0},
        fill_color={'value': 'red'},
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha={'value': 1.0},
        line_cap='butt',
        line_color={'value': 'red'},
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width={'value': 1},
        name=None,
        radius=None,
        radius_dimension='x',
        size={'value': 10, 'units': 'screen'},
        subscribed_events=[],
        tags=[],
        x={'value': 0.04185751513638254},
        y={'value': 0.561837822186348}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph=bokeh.models.markers.Circle(
        id='5cf41f76-8f6a-4eec-9ad1-dd6119e56c4d',
        angle={'value': 0.0, 'units': 'rad'},
        fill_alpha={'value': 0.1},
        fill_color={'value': '#1f77b4'},
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha={'value': 0.1},
        line_cap='butt',
        line_color={'value': '#1f77b4'},
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width={'value': 1},
        name=None,
        radius=None,
        radius_dimension='x',
        size={'value': 10, 'units': 'screen'},
        subscribed_events=[],
        tags=[],
        x={'value': 0.04185751513638254},
        y={'value': 0.561837822186348}),
    selection_glyph=None,
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='c4ea1e89-a805-419e-89ce-beb42e914fea',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='beef3413-9a47-45e0-acfe-01dc7eee71b0',
            callback=None,
            data={},
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            selected=bokeh.models.selections.Selection(
                id='4758660c-38fd-48b3-abce-2be50a0c6f3c',
                indices=[],
                js_event_callbacks={},
                js_property_callbacks={},
                line_indices=[],
                multiline_indices={},
                name=None,
                subscribed_events=[],
                tags=[]),
            selection_policy=bokeh.models.selections.UnionRenderers(
                id='ea80e412-24a1-49f5-910e-7e419d7a3e8b',
                js_event_callbacks={},
                js_property_callbacks={},
                name=None,
                subscribed_events=[],
                tags=[]),
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    visible=True,
    x_range_name='default',
    y_range_name='default')

In [38]:
show(p)

In [18]:
print('utter_ask_price', '-->', domain.index_for_action('utter_ask_price'))
print('utter_ask_details', '-->', domain.index_for_action('utter_ask_details'))
print('utter_ask_people', '-->', domain.index_for_action('utter_ask_people'))
print('utter_explain_details_hotel', '-->', domain.index_for_action('utter_explain_details_hotel'))
print('utter_explain_people_hotel', '-->', domain.index_for_action('utter_explain_people_hotel'))
print('utter_explain_price_hotel', '-->', domain.index_for_action('utter_explain_price_hotel'))
print('utter_ask_location', '-->', domain.index_for_action('utter_ask_location'))
print('utter_explain_location_hotel', '-->', domain.index_for_action('utter_explain_location_hotel'))
print('utter_ask_startdate', '-->', domain.index_for_action('utter_ask_startdate'))
print('utter_suggest_hotel', '-->', domain.index_for_action('utter_suggest_hotel'))
print('action_search_hotel', '-->', domain.index_for_action('action_search_hotel'))
print('action_listen', '-->', domain.index_for_action('action_listen'))

utter_ask_price --> 6
utter_ask_details --> 28
utter_ask_people --> 4
utter_explain_details_hotel --> 14
utter_explain_people_hotel --> 19
utter_explain_price_hotel --> 21
utter_ask_location --> 5
utter_explain_location_hotel --> 17
utter_ask_startdate --> 7
utter_suggest_hotel --> 12
action_search_hotel --> 42
action_listen --> 0


In [19]:
utter_price = embedding_from_tracker("#generate story\n{}".format(story_block), bot=True)[0,-1,5,:]
utter_price.shape

(20,)

In [20]:
print(utter_price.shape)
x = pca.transform(utter_price.reshape(1,-1))
p.circle(x[0][0], x[0][1], color='black')

(20,)


GlyphRenderer(id='1db745d9-cfee-44a7-8a51-a238a358f252', ...)

In [21]:
show(p)

In [22]:
from collections import namedtuple

In [23]:
Point = namedtuple('Point', ['x', 'y'])
p = Point(11, y=22)
p._asdict()

OrderedDict([('x', 11), ('y', 22)])

In [24]:
Point3D = namedtuple('Point3D', Point._fields + ('z',))

In [25]:
p_asdict = p._asdict()
p_asdict['z'] = 3
p_asdict

OrderedDict([('x', 11), ('y', 22), ('z', 3)])

In [26]:
p3D = Point3D(**p_asdict)
p3D

Point3D(x=11, y=22, z=3)

In [27]:
x ={
        "day": "20",
        "month": "06",
        "year": "2018"
    }

In [28]:
x

{'day': '20', 'month': '06', 'year': '2018'}

In [29]:
def value_touched_dict(value):
    if isinstance(value, dict) and value.get("day"):
        value["value1"] = value.pop("day")
        value["value2"] = value.pop("month")
        value["value3"] = value.pop("year")
    return {"value": value,
            "touched": True}
value_touched_dict(x)

{'touched': True, 'value': {'value1': '20', 'value2': '06', 'value3': '2018'}}

In [30]:
value_touched_dict('x')

{'touched': True, 'value': 'x'}